In [1]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/heather/code/loicmorel/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [2]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [4]:
file = 'data/ohlcv_DOTUSDT_1h_from_1597791600000_to_1651590000000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2020-08-18 23:00:00,2.0900,4.4400,2.0000,3.1000,6039881.97,2020-08-18 23:59:59.999000064,18492.958333
1,2020-08-19 00:00:00,3.1000,3.1400,2.9738,3.1050,1554670.37,2020-08-19 00:59:59.999000064,18493.000000
2,2020-08-19 01:00:00,3.1050,3.1299,3.0584,3.1063,813079.44,2020-08-19 01:59:59.999000064,18493.041667
3,2020-08-19 02:00:00,3.1082,3.1124,2.9000,2.9628,1472769.86,2020-08-19 02:59:59.999000064,18493.083333
4,2020-08-19 03:00:00,2.9631,3.0032,2.6400,2.8850,1768853.60,2020-08-19 03:59:59.999000064,18493.125000
...,...,...,...,...,...,...,...,...
14922,2022-05-03 11:00:00,14.9700,15.2000,14.9000,15.1500,181452.93,2022-05-03 11:59:59.999000064,19115.458333
14923,2022-05-03 12:00:00,15.1500,15.1700,15.0200,15.0200,101209.35,2022-05-03 12:59:59.999000064,19115.500000
14924,2022-05-03 13:00:00,15.0300,15.0600,14.8600,14.9800,229277.95,2022-05-03 13:59:59.999000064,19115.541667
14925,2022-05-03 14:00:00,14.9800,15.0400,14.8700,14.9900,162995.81,2022-05-03 14:59:59.999000064,19115.583333


In [5]:
from app.indicators import find_swings
data_ = find_swings(data)

In [6]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [7]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,4.44,2.0000,4.44,0.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
1,3.14,2.9738,3.14,NaN,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
2,4.44,0.0000,4.44,-1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
3,4.44,0.0000,4.44,-1.0,1,False,True,0.00,4.44,0.00,4.44,False,False,True,False
4,4.44,0.0000,4.44,-1.0,2,False,True,0.00,4.44,0.00,0.00,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14922,15.21,14.8700,14.87,1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
14923,15.21,14.8700,14.87,1.0,1,True,False,14.87,0.00,14.87,0.00,True,False,False,False
14924,15.20,14.8700,15.20,-1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
14925,15.20,14.8700,15.20,-1.0,1,False,True,0.00,15.20,0.00,15.20,False,False,False,True


In [8]:
data__=data__*1

In [9]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,4.44,2.0000,4.44,0.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
1,3.14,2.9738,3.14,NaN,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
2,4.44,0.0000,4.44,-1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
3,4.44,0.0000,4.44,-1.0,1,0,1,0.00,4.44,0.00,4.44,0,0,1,0
4,4.44,0.0000,4.44,-1.0,2,0,1,0.00,4.44,0.00,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14922,15.21,14.8700,14.87,1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
14923,15.21,14.8700,14.87,1.0,1,1,0,14.87,0.00,14.87,0.00,1,0,0,0
14924,15.20,14.8700,15.20,-1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
14925,15.20,14.8700,15.20,-1.0,1,0,1,0.00,15.20,0.00,15.20,0,0,0,1


In [10]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [11]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [12]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2020-08-18 23:00:00,2.0900,4.4400,2.0000,3.1000,6039881.97,0,0,0,0
1,2020-08-19 00:00:00,3.1000,3.1400,2.9738,3.1050,1554670.37,0,0,0,0
2,2020-08-19 01:00:00,3.1050,3.1299,3.0584,3.1063,813079.44,0,0,0,0
3,2020-08-19 02:00:00,3.1082,3.1124,2.9000,2.9628,1472769.86,0,0,1,0
4,2020-08-19 03:00:00,2.9631,3.0032,2.6400,2.8850,1768853.60,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
14922,2022-05-03 11:00:00,14.9700,15.2000,14.9000,15.1500,181452.93,0,0,0,0
14923,2022-05-03 12:00:00,15.1500,15.1700,15.0200,15.0200,101209.35,1,0,0,0
14924,2022-05-03 13:00:00,15.0300,15.0600,14.8600,14.9800,229277.95,0,0,0,0
14925,2022-05-03 14:00:00,14.9800,15.0400,14.8700,14.9900,162995.81,0,0,0,1


In [13]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [14]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2020-08-18 23:00:00,2.0900,4.4400,2.0000,3.1000,6039881.97,0,0,0,0,1
1,2020-08-19 00:00:00,3.1000,3.1400,2.9738,3.1050,1554670.37,0,0,0,0,1
2,2020-08-19 01:00:00,3.1050,3.1299,3.0584,3.1063,813079.44,0,0,0,0,1
3,2020-08-19 02:00:00,3.1082,3.1124,2.9000,2.9628,1472769.86,0,0,1,0,0
4,2020-08-19 03:00:00,2.9631,3.0032,2.6400,2.8850,1768853.60,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
14922,2022-05-03 11:00:00,14.9700,15.2000,14.9000,15.1500,181452.93,0,0,0,0,1
14923,2022-05-03 12:00:00,15.1500,15.1700,15.0200,15.0200,101209.35,1,0,0,0,0
14924,2022-05-03 13:00:00,15.0300,15.0600,14.8600,14.9800,229277.95,0,0,0,0,1
14925,2022-05-03 14:00:00,14.9800,15.0400,14.8700,14.9900,162995.81,0,0,0,1,0


In [15]:
df.set_index('o_ts',inplace=True)

In [16]:
df.to_csv('data/transformed_data/DOTUSDT_1h_transformed.csv')

In [17]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     11403
    1   0   0   0      1024
    0   0   1   0      1011
1   0   0   0   0       781
0   0   1   0   0       708
dtype: int64

In [18]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [19]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [20]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [21]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [22]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [23]:
X, y = get_X_y(df, 1000, 21)
print(X.shape)
print(y.shape)

(1000, 20, 10)
(1000, 5)


In [24]:
assert np.isnan(X).sum() == 0

In [25]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [26]:
X_train, y_train = get_X_y(data_train, 1000, 21)
X_test, y_test = get_X_y(data_test, 100, 21)

In [27]:
assert X_train.shape == (1000, 20, 10)

In [28]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [29]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [30]:
model = init_model()
model.summary()

2022-05-06 13:07:44.333262: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 13:07:44.359375: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [32]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [33]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75
13/13 [==============================] - 2s 44ms/step - loss: 3.0096 - accuracy: 0.6325 - val_loss: 1.1297 - val_accuracy: 0.6750
Epoch 2/75
13/13 [==============================] - 0s 12ms/step - loss: 2.6722 - accuracy: 0.6375 - val_loss: 1.2370 - val_accuracy: 0.4150
Epoch 3/75
13/13 [==============================] - 0s 11ms/step - loss: 2.5512 - accuracy: 0.5675 - val_loss: 1.1155 - val_accuracy: 0.5150
Epoch 4/75
13/13 [==============================] - 0s 11ms/step - loss: 2.4848 - accuracy: 0.5350 - val_loss: 1.0374 - val_accuracy: 0.5400
Epoch 5/75
13/13 [==============================] - 0s 12ms/step - loss: 2.3914 - accuracy: 0.5575 - val_loss: 1.1356 - val_accuracy: 0.4050
Epoch 6/75
13/13 [==============================] - 0s 12ms/step - loss: 2.3442 - accuracy: 0.5387 - val_loss: 1.1473 - val_accuracy: 0.4200
Epoch 7/75
13/13

In [34]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 1.0604 - accuracy: 0.4500


[1.0603687763214111, 0.44999998807907104]

In [35]:
np.round(model.predict_proba(X_test),2)

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.42, 0.12, 0.  , 0.09, 0.37],
       [0.53, 0.07, 0.  , 0.1 , 0.29],
       [0.45, 0.09, 0.  , 0.16, 0.29],
       [0.02, 0.23, 0.28, 0.2 , 0.25],
       [0.06, 0.3 , 0.  , 0.01, 0.63],
       [0.  , 0.02, 0.68, 0.09, 0.21],
       [0.  , 0.02, 0.16, 0.41, 0.4 ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.55, 0.08, 0.  , 0.12, 0.25],
       [0.33, 0.13, 0.01, 0.17, 0.36],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.6 , 0.06, 0.  , 0.09, 0.25],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.01, 0.01, 0.54, 0.44],
       [0.01, 0.26, 0.08, 0.4 , 0.25],
       [0.42, 0.12, 0.01, 0.15, 0.3 ],
       [0.04, 0.22, 0.16, 0.29, 0.29],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.01, 0.  , 0.  , 0.99],
       [0.  , 0.02, 0.17, 0.44, 0.37],
       [0.42, 0.15, 0.  , 0.08, 0.36],
       [0.01, 0.27, 0.13, 0.37, 0.22],
       [0.09, 0.27, 0.01, 0.03, 0.59],
       [0.01, 0.29, 0.24, 0.26, 0.2 ],
       [0.06, 0.48, 0.  , 0.02, 0.44],
       [0.39, 0.13, 0.01,

In [36]:
model.save('./models/RNN_DOTUSDT_1h.h5')